In [24]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from consts_and_funcs import *

# 计算给定参数的屏蔽时间函数
def shield_time(theta, v, t_deploy, delay):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    
    # 部署位置
    deploy_pos = FY1_INIT + drone_vel * t_deploy
    
    # 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    # 云中心函数
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    # 内半径和外半径的平均屏蔽时间
    calculate_shield_time = get_calculate_shield_time(cloud_center, explode_t, 1)
    inner = calculate_shield_time(INNER_RADIUS)
    outer = calculate_shield_time(OUTER_RADIUS)
    return (inner + outer) / 2

def min_distance(theta, v, t_deploy, delay):
    # 无人机方向（xy平面，theta是从-x轴的角度）
    dir_unit = np.array([-np.cos(theta), np.sin(theta), 0])
    drone_vel = v * dir_unit
    
    # 部署位置
    deploy_pos = FY1_INIT + drone_vel * t_deploy
    
    # 爆炸位置（烟雾弹具有无人机的水平速度，加上重力）
    explode_pos = deploy_pos + drone_vel * delay + np.array([0, 0, -0.5 * G * delay**2])
    explode_t = t_deploy + delay
    
    # 云中心函数
    def cloud_center(t):
        tau = t - explode_t
        if tau < 0:
            return explode_pos + np.array([0, 0, -ESCAPE_VEL * tau])
        if tau > DURATION:
            return explode_pos + np.array([0, 0, -SINK_SPEED * DURATION]) + np.array([0, 0, -ESCAPE_VEL * (tau - DURATION)])
        else:
            return explode_pos + np.array([0, 0, -SINK_SPEED * tau])
    
    return get_min_distance(cloud_center, explode_t, 1)

def objective0(params):
    return min_distance(*params)

# 目标函数（最小化负屏蔽时间以最大化）
def objective(params):
    return -shield_time(*params)

if __name__ == '__main__':
    # 边界：theta（-360°到360°），v（70-140），t_deploy（>=0），delay（>=0）
    bounds = [(-np.pi/6, np.pi/6), (70, 140), (0.1, 10), (0.1, 10)]
    iteration = 0
    
    def callback(xk, convergence):
        global iteration
        iteration += 1
        if iteration % 10 == 0:  # Print every 10 iterations
            current_time = -objective(xk)
            theta, v, t_deploy, delay = xk
            print(f"Iteration {iteration}: Best shielding time= {current_time:.3f} s, "
                  f"theta = {np.degrees(theta) % 360:.3f}°, v = {v:.3f} m/s, "
                  f"t_deploy = {t_deploy:.3f} s, delay = {delay:.3f} s")
    
    # 使用SLSQP从differential_evolution结果进行局部优化，优化最短距离，搞10种结果
    result0 = []
    for _ in range(10):
        a_result0 = differential_evolution(
            objective0,
            bounds,
            strategy='best1bin',
            maxiter=10,
            popsize=10,
            tol=1e-3,
            callback=callback,
            init='random',
            disp=False,
            polish=False
        )
        result0.append(a_result0)
    
    # 提取优化参数
    result = []
    for a_result0 in result0:
        # 使用L-BFGS-B细优化
        a_result = minimize(
            objective,
            a_result0.x,
            bounds=bounds,
            method='SLSQP',
            options={'maxiter': 200}
        )
        result.append(a_result)
    
    max_index = np.array([-a.fun for a in result]).argmax()
    opt_theta, opt_v, opt_t_deploy, opt_delay = result[max_index].x
    opt_time = -result[max_index].fun
    
    # 计算位置
    dir_unit = np.array([-np.cos(opt_theta), np.sin(opt_theta), 0])
    drone_vel = opt_v * dir_unit
    deploy_pos = FY1_INIT + drone_vel * opt_t_deploy
    explode_pos = deploy_pos + drone_vel * opt_delay + np.array([0, 0, -0.5 * G * opt_delay**2])
    theta_deg = np.degrees(opt_theta) % 360
    
    # 输出结果
    print(f"优化参数：")
    print(f"飞行方向 (theta)：{opt_theta:.6f} rad ({theta_deg:.3f}°)")
    print(f"飞行速度：{opt_v:.3f} m/s")
    print(f"无人机速度向量：{drone_vel}")
    print(f"部署时间：{opt_t_deploy:.3f} s")
    print(f"爆炸延迟：{opt_delay:.3f} s")
    print(f"部署位置：{deploy_pos}")
    print(f"爆炸位置：{explode_pos}")
    print(f"有效屏蔽时间（平均）：{opt_time:.3f} s")

ModuleNotFoundError: No module named 'consts_and_funcs'